In [ ]:
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

from pdflight import in_task, shift_datetime

%matplotlib inline

In [ ]:
with open('store.pkl', 'rb') as f:
    stored = pickle.load(f)
    md, fl, th = stored['md'], stored['fl'], stored['th']

In [ ]:
th

In [ ]:
tt = th.loc[in_task(th, md)]
tt

In [ ]:
df = pd.DataFrame(index=tt.index)
for i in tt.index:
    df.at[i, "v"] = (
        fl.loc[shift_datetime(i, tt.loc[i])].alt - fl.loc[i].alt
    ) / (tt.loc[i].total_seconds())

In [ ]:
cc = md[["pilot", "points"]].join(df.v.groupby("flight").mean(), how="inner")

In [ ]:
sns.regplot(data=cc, x='v', y='points', line_kws={"color": "red"})
plt.show()

In [ ]:
res = stats.linregress(cc.v, cc.points)
res